# OpenAI Using API Assistant Files


This notebook demonstrate the ability of AI to convert the student report to Conferenct Paper. 

In [1]:
from openai import OpenAI
from dotenv import find_dotenv, load_dotenv

In [2]:
load_dotenv()
client = OpenAI()  

In [3]:
# thesisName = "eyeThesis"
thesisName = "fonThesis"

In [5]:
# helper function to get completions from OpenAI
def get_completion(prompt, model="gpt-3.5-turbo"):
    messages = [{"role": "user", "content": prompt}]
    response = client.chat.completions.create(
        model=model,
        messages=messages,
        temperature=0,
        max_tokens= 2000
    )
    return response.choices[0].message.content

In [ ]:
# Create Assitant
# assistant = client.beta.assistants.create(
#   name="Paper Writing Assistant",
#   instructions="You are an expert financial analyst. Use you knowledge base to answer questions about audited financial statements.",
#   model="gpt-3.5-turbo",
#   tools=[{"type": "file_search"}],
# )
# assistant_id = assistant.id
# print(assistant_id)

In [6]:
assistant_id = 'asst_ncWicyO0j4TsiOsdtuDpldC6'

In [ ]:
# # Create a vector store called "Paper Data"
# vector_store = client.beta.vector_stores.create(name="Paper Data")
 
# # Ready the files for upload to OpenAI 
# file_paths = ["/Users/taweesaksamanchuen/Dev/ApiAssistant/book/eyeThesis.pdf", "/Users/taweesaksamanchuen/Dev/ApiAssistant/book/fonThesis.pdf"]
# file_streams = [open(path, "rb") for path in file_paths]
 
# # Use the upload and poll SDK helper to upload the files, add them to the vector store,
# # and poll the status of the file batch for completion.
# file_batch = client.beta.vector_stores.file_batches.upload_and_poll(
#   vector_store_id=vector_store.id, files=file_streams
# )
 
# # You can print the status and the file counts of the batch to see the result of this operation. 
# print(file_batch.status)
# print(file_batch.file_counts)

In [ ]:
# Attach the vector store to the assistant
# assistant = client.beta.assistants.update(
#   assistant_id=assistant_id,
#   tool_resources={"file_search": {"vector_store_ids": [vector_store.id]}},
# )


In [7]:
# Upload the user provided file to OpenAI
# message_file = client.files.create(
#   file=open("edgar/aapl-10k.pdf", "rb"), purpose="assistants"
# )
 
# Create a thread and attach the file to the message
thread = client.beta.threads.create(
  messages=[
    {
      "role": "user",
      "content": "Study the PDF file and summarize the key points.",
      # Attach the new file to the message.
    #   "attachments": [
    #     { "file_id": message_file.id, "tools": [{"type": "file_search"}] }
    #  ],
    }
  ]
)
 
# The thread now has a vector store with that file in its tool resources.
print(thread.tool_resources.file_search)
thread_id = thread.id
print(thread_id)

None
thread_P3qOUPioTqtNSQ0m674vdA0I


In [ ]:
# #current thread id 
# thread_id = 'thread_pUGOPAakwhHWwgKwF9wztzl8'
# #delete thread
# client.beta.threads.delete(thread_id)




In [8]:
from typing_extensions import override
from openai import AssistantEventHandler
 
class EventHandler(AssistantEventHandler):
    @override
    def on_text_created(self, text) -> None:
        print(f"\nassistant > ", end="", flush=True)

    @override
    def on_tool_call_created(self, tool_call):
        print(f"\nassistant > {tool_call.type}\n", flush=True)

    @override
    def on_message_done(self, message) -> None:
        # print a citation to the file searched
        message_content = message.content[0].text
        annotations = message_content.annotations
        citations = []
        for index, annotation in enumerate(annotations):
            message_content.value = message_content.value.replace(
                annotation.text, f"[{index}]"
            )
            if file_citation := getattr(annotation, "file_citation", None):
                cited_file = client.files.retrieve(file_citation.file_id)
                citations.append(f"[{index}] {cited_file.filename}")

        print(message_content.value)
        print("\n".join(citations))




In [9]:
def ask_assistant(instuction_message, user_message):
    message = client.beta.threads.messages.create(
    thread_id=thread_id,
    role="user",
    content= user_message,
    )
    with client.beta.threads.runs.stream(
        thread_id = thread_id,
        assistant_id = assistant_id,
        instructions= instuction_message,
        event_handler=EventHandler(),
    ) as stream:
        stream.until_done()
        return stream.get_final_messages()

In [11]:
instuction_message = "Act as Academic Writer to create a paper from the given topic."
user_message = f"""from {thesisName},  what is the key point of the PDF file?"""
ask_assistant(instuction_message, user_message)


assistant > file_search


assistant > The key point of the PDF file "RETAIL BUSINESS CHATBOT IMPLEMENTATION WITH DIALOGFLOW AND IBM WATSON" is the evaluation of chatbot platforms based on confidence values, intent classification, accuracy, and user perceptions. The study suggests that aesthetic and minimalist design should be given less importance compared to intent and context accuracy when evaluating chatbot platforms. It emphasizes the significance of core functionality and accuracy in designing and evaluating chatbots, aiming to enhance flow accuracy, provide helpful error messages, and offer user control freedom. The findings provide practical implications for designing user-friendly interfaces that prioritize seamless user experiences[0].
[0] fonThesis.pdf


[Message(id='msg_VbDxOPMmtsGOG9CFaFwPjJLO', assistant_id='asst_ncWicyO0j4TsiOsdtuDpldC6', attachments=[], completed_at=1714455893, content=[TextContentBlock(text=Text(annotations=[FileCitationAnnotation(end_index=724, file_citation=FileCitation(file_id='file-gfeZfARaGZbR7ow4qtETdRu8', quote=None), start_index=712, text='【5:1†source】', type='file_citation')], value='The key point of the PDF file "RETAIL BUSINESS CHATBOT IMPLEMENTATION WITH DIALOGFLOW AND IBM WATSON" is the evaluation of chatbot platforms based on confidence values, intent classification, accuracy, and user perceptions. The study suggests that aesthetic and minimalist design should be given less importance compared to intent and context accuracy when evaluating chatbot platforms. It emphasizes the significance of core functionality and accuracy in designing and evaluating chatbots, aiming to enhance flow accuracy, provide helpful error messages, and offer user control freedom. The findings provide practical implications 